In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()

client = OpenAI(api_key=os.environ['API_KEY'])

MODEL_NAME = "gpt-4o-mini"

In [2]:
df = pd.read_excel('sample01.xlsx', sheet_name='Sheet1')
df.head()

,Date,Product,Region,Quantity,UnitPrice,Sales
0,2024-04-12,Tablet,North,5,870,4350
1,2024-06-28,PC,East,4,536,2144
2,2024-04-02,Accessory,South,8,122,976
3,2024-01-15,Tablet,North,7,664,4648
4,2024-04-16,Accessory,South,2,776,1552


In [3]:
data_text = df.astype(str)
type(df)

pandas.core.frame.DataFrame

In [4]:
df["Month"] = df["Date"].dt.to_period("M").dt.to_timestamp()
monthly_sales = df.groupby("Month", as_index=False)["Sales"].sum()
df["Day"] = df["Date"].dt.to_period("D").dt.to_timestamp()
daily_sales = df.groupby("Day", as_index=False)["Sales"].sum()
df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()
weekly_sales = df.groupby("Week", as_index=False)["Sales"].sum()
avg_unitprice_by_product = df.groupby("Product", as_index=False)["UnitPrice"].mean()

region_product_sales = df.groupby(["Region", "Product"], as_index=False)["Sales"].sum()
idx = region_product_sales.groupby("Region")["Sales"].idxmax()
top_product_by_region = region_product_sales.loc[idx].reset_index(drop=True) 
top_product_by_region


,Region,Product,Sales
0,East,Accessory,2228
1,North,Tablet,8998
2,South,Accessory,2528
3,West,Phone,2008


In [5]:
context = {
    "monthly_sales": monthly_sales.to_dict(orient="records"),
    "avg_unitprice_by_product": avg_unitprice_by_product.to_dict(orient="records"),
    "top_product_by_region": top_product_by_region.to_dict(orient="records"),
}

context

{'monthly_sales': [{'Month': Timestamp('2024-01-01 00:00:00'), 'Sales': 6876},
  {'Month': Timestamp('2024-03-01 00:00:00'), 'Sales': 1365},
  {'Month': Timestamp('2024-04-01 00:00:00'), 'Sales': 8334},
  {'Month': Timestamp('2024-05-01 00:00:00'), 'Sales': 488},
  {'Month': Timestamp('2024-06-01 00:00:00'), 'Sales': 2144}],
 'avg_unitprice_by_product': [{'Product': 'Accessory', 'UnitPrice': 424.75},
  {'Product': 'PC', 'UnitPrice': 536.0},
  {'Product': 'Phone', 'UnitPrice': 183.0},
  {'Product': 'Tablet', 'UnitPrice': 782.3333333333334}],
 'top_product_by_region': [{'Region': 'East',
   'Product': 'Accessory',
   'Sales': 2228},
  {'Region': 'North', 'Product': 'Tablet', 'Sales': 8998},
  {'Region': 'South', 'Product': 'Accessory', 'Sales': 2528},
  {'Region': 'West', 'Product': 'Phone', 'Sales': 2008}]}

In [6]:
prompt_text = f"""
以下は売上データの集計結果です:
{context}

これをもとに全体の傾向を3点だけ日本語で要約してください。
出力はJSON形式で: {{"analysis": ["...", "...", "..."]}}
"""

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 結果を取得
result = response.choices[0].message.content.strip()


In [10]:
result


'{"analysis": ["2024年の月別売上は1月に高い数字を記録し、他の月と比較して大きなバラツキが見られる。", "製品別の平均単価では、タブレットが最も高価で、次いでPCとアクセサリーが続く。", "地域別の売上では、北部地区でタブレットが最も売上が高く、南部地域ではアクセサリーが人気を集めている。"]}'

In [11]:
import json

# ① JSON文字列 → dict に変換
payload = json.loads(result)  # <= ここが重要

# ② analysis を DataFrame に
analysis_list = payload.get("analysis", [])
df_analysis = pd.DataFrame({"Analysis": analysis_list})

with pd.ExcelWriter("output.xlsx") as writer:
    monthly_sales.to_excel(writer, sheet_name="MonthlySales", index=False)
    avg_unitprice_by_product.to_excel(writer, sheet_name="AvgUnitPriceByProduct", index=False)
    top_product_by_region.to_excel(writer, sheet_name="TopProductByRegion", index=False)
    df_analysis.to_excel(writer, sheet_name="Analysis", index=False)